In [ ]:
# default_exp models.RNNPlus

# RNNPlus

> These are RNN, LSTM and GRU PyTorch implementations created by Ignacio Oguiza - oguiza@gmail.com based on:

In [ ]:
#export
from tsai.imports import *
from tsai.utils import *
from tsai.data.core import *
from tsai.models.layers import *

In [ ]:
#hide
# example
# rnn = nn.LSTM(10, 20, 2)
# input = torch.randn(5, 3, 10)
# h0 = torch.randn(2, 3, 20)
# c0 = torch.randn(2, 3, 20)
# output, (hn, cn) = rnn(input, (h0, c0))

In [ ]:
#export
class _RNNPlus_Base(Module):
    def __init__(self, c_in, c_out, seq_len=None, hidden_size=100, n_layers=1, bias=True, rnn_dropout=0, bidirectional=False, fc_dropout=0., 
                 last_step=True, bn=False, custom_head=None, y_range=None, **kwargs):
        if not last_step: assert seq_len, 'you need to enter a seq_len to use flatten=True'
        self.rnn = self._cell(c_in, hidden_size, num_layers=n_layers, bias=bias, batch_first=True, dropout=rnn_dropout, bidirectional=bidirectional)
        self.transpose = Transpose(-1, -2, contiguous=True)
        
        # Head
        self.head_nf = hidden_size * (1 + bidirectional)
        if custom_head: self.head = custom_head(self.head_nf, c_out, seq_len) # custom head must have all required kwargs
        else: self.head = self.create_head(self.head_nf, c_out, seq_len, last_step=last_step, fc_dropout=fc_dropout, bn=bn, y_range=y_range)

    def forward(self, x): 
        x = x.transpose(2,1)                               # [batch_size x n_vars x seq_len] --> [batch_size x seq_len x n_vars]
        output, _ = self.rnn(x)                            # [batch_size x seq_len x hidden_size * (1 + bidirectional)]
        output = self.transpose(output)                    # [batch_size x hidden_size * (1 + bidirectional) x seq_len]
        return self.head(output)
    
    def create_head(self, nf, c_out, seq_len, last_step=True, fc_dropout=0., bn=False, y_range=None):
        if last_step: 
            layers = [LastStep()]
        else: 
            layers = [Flatten()]
            nf *= seq_len
        layers += [LinBnDrop(nf, c_out, bn=bn, p=fc_dropout)]
        if y_range: layers += [SigmoidRange(*y_range)]
        return nn.Sequential(*layers)

    
class RNNPlus(_RNNPlus_Base):
    _cell = nn.RNN
    
class LSTMPlus(_RNNPlus_Base):
    _cell = nn.LSTM
    
class GRUPlus(_RNNPlus_Base):
    _cell = nn.GRU

In [ ]:
bs = 16
c_in = 3
seq_len = 12
c_out = 2
xb = torch.rand(bs, c_in, seq_len)
test_eq(RNNPlus(c_in, c_out)(xb).shape, [bs, c_out])
test_eq(RNNPlus(c_in, c_out, hidden_size=100, n_layers=2, bias=True, rnn_dropout=0.2, bidirectional=True, fc_dropout=0.5)(xb).shape, [bs, c_out])
test_eq(LSTMPlus(c_in, c_out, hidden_size=100, n_layers=2, bias=True, rnn_dropout=0.2, bidirectional=True, fc_dropout=0.5)(xb).shape, [bs, c_out])
test_eq(GRUPlus(c_in, c_out, hidden_size=100, n_layers=2, bias=True, rnn_dropout=0.2, bidirectional=True, fc_dropout=0.5)(xb).shape, [bs, c_out])
test_eq(RNNPlus(c_in, c_out, seq_len, flatten=True)(xb).shape, [bs, c_out])
test_eq(RNNPlus(c_in, c_out, seq_len, flatten=True)(xb).shape, [bs, c_out])
test_eq(RNNPlus(c_in, c_out, seq_len, hidden_size=100, n_layers=2, bias=True, rnn_dropout=0.2, bidirectional=True, fc_dropout=0.5, flatten=True)(xb).shape, 
        [bs, c_out])
test_eq(LSTMPlus(c_in, c_out, seq_len, flatten=True)(xb).shape, [bs, c_out])
test_eq(GRUPlus(c_in, c_out, seq_len, last_step=False, flatten=True)(xb).shape, [bs, c_out])

In [ ]:
bs = 16
c_in = 3
seq_len = 12
c_out = 2
xb = torch.rand(bs, c_in, seq_len)
custom_head = partial(create_mlp_head, fc_dropout=0.5)
test_eq(LSTMPlus(c_in, c_out, seq_len, flatten=True, custom_head=custom_head)(xb).shape, [bs, c_out])
custom_head = partial(create_pool_head, concat_pool=True, fc_dropout=0.5)
test_eq(LSTMPlus(c_in, c_out, seq_len, last_step=False, flatten=False, custom_head=custom_head)(xb).shape, [bs, c_out])
custom_head = partial(create_pool_plus_head, fc_dropout=0.5)
test_eq(LSTMPlus(c_in, c_out, seq_len, last_step=False, flatten=False, custom_head=custom_head)(xb).shape, [bs, c_out])
custom_head = partial(create_conv_head)
test_eq(LSTMPlus(c_in, c_out, seq_len, last_step=False, flatten=False, custom_head=custom_head)(xb).shape, [bs, c_out])

In [ ]:
from tsai.data.all import *
from tsai.models.utils import *
dsid = 'NATOPS' 
bs = 16
X, y, splits = get_UCR_data(dsid, return_split=False)
tfms  = [None, [Categorize()]]
dls = get_ts_dls(X, y, tfms=tfms, splits=splits, bs=bs)

In [ ]:
model = build_ts_model(LSTMPlus, dls=dls)
print(model.head)
learn = Learner(dls, model,  metrics=accuracy)
learn.fit_one_cycle(1, 3e-3)

Sequential(
  (0): LastStep()
  (1): LinBnDrop(
    (0): Linear(in_features=100, out_features=6, bias=True)
  )
)


epoch,train_loss,valid_loss,accuracy,time
0,1.822123,1.796828,0.172222,00:02


In [ ]:
model = LSTMPlus(dls.vars, dls.c, dls.len, last_step=False, flatten=True)
print(model.head)
learn = Learner(dls, model,  metrics=accuracy)
learn.fit_one_cycle(1, 3e-3)

Sequential(
  (0): Flatten(full=False)
  (1): LinBnDrop(
    (0): Linear(in_features=5100, out_features=6, bias=True)
  )
)


epoch,train_loss,valid_loss,accuracy,time
0,1.293266,0.815523,0.694444,00:02


In [ ]:
custom_head = partial(create_pool_head, concat_pool=True)
model = LSTMPlus(dls.vars, dls.c, dls.len, last_step=False, flatten=False, custom_head=custom_head)
print(model.head)
learn = Learner(dls, model,  metrics=accuracy)
learn.fit_one_cycle(1, 3e-3)

Sequential(
  (0): GACP1d(
    (gacp): AdaptiveConcatPool1d(
      (ap): AdaptiveAvgPool1d(output_size=1)
      (mp): AdaptiveMaxPool1d(output_size=1)
    )
    (flatten): Flatten(full=False)
  )
  (1): LinBnDrop(
    (0): Linear(in_features=200, out_features=6, bias=True)
  )
)


epoch,train_loss,valid_loss,accuracy,time
0,1.738512,1.644175,0.472222,00:02


In [ ]:
custom_head = partial(create_pool_plus_head, concat_pool=True)
model = LSTMPlus(dls.vars, dls.c, dls.len, last_step=False, flatten=False, custom_head=custom_head)
print(model.head)
learn = Learner(dls, model,  metrics=accuracy)
learn.fit_one_cycle(1, 3e-3)

Sequential(
  (0): AdaptiveConcatPool1d(
    (ap): AdaptiveAvgPool1d(output_size=1)
    (mp): AdaptiveMaxPool1d(output_size=1)
  )
  (1): Flatten(full=False)
  (2): BatchNorm1d(200, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Linear(in_features=200, out_features=512, bias=False)
  (4): ReLU(inplace=True)
  (5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): Linear(in_features=512, out_features=6, bias=False)
)


epoch,train_loss,valid_loss,accuracy,time
0,0.963994,1.583249,0.594444,00:02


In [ ]:
m = RNNPlus(c_in, c_out, seq_len, hidden_size=100,n_layers=2,bidirectional=True,rnn_dropout=.5,fc_dropout=.5, flatten=True)
print(m)
print(total_params(m))
m(xb).shape

RNNPlus(
  (rnn): RNN(3, 100, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (transpose): Transpose(dims=-1, -2).contiguous()
  (head): Sequential(
    (0): LastStep()
    (1): LinBnDrop(
      (0): Dropout(p=0.5, inplace=False)
      (1): Linear(in_features=200, out_features=2, bias=True)
    )
  )
)
(81802, True)


torch.Size([16, 2])

In [ ]:
m = LSTMPlus(c_in, c_out, seq_len, hidden_size=100,n_layers=2,bidirectional=True,rnn_dropout=.5,fc_dropout=.5, flatten=True)
print(m)
print(total_params(m))
m(xb).shape

LSTMPlus(
  (rnn): LSTM(3, 100, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (transpose): Transpose(dims=-1, -2).contiguous()
  (head): Sequential(
    (0): LastStep()
    (1): LinBnDrop(
      (0): Dropout(p=0.5, inplace=False)
      (1): Linear(in_features=200, out_features=2, bias=True)
    )
  )
)
(326002, True)


torch.Size([16, 2])

In [ ]:
m = GRUPlus(c_in, c_out, seq_len, hidden_size=100,n_layers=2,bidirectional=True,rnn_dropout=.5,fc_dropout=.5, flatten=True)
print(m)
print(total_params(m))
m(xb).shape

GRUPlus(
  (rnn): GRU(3, 100, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (transpose): Transpose(dims=-1, -2).contiguous()
  (head): Sequential(
    (0): LastStep()
    (1): LinBnDrop(
      (0): Dropout(p=0.5, inplace=False)
      (1): Linear(in_features=200, out_features=2, bias=True)
    )
  )
)
(244602, True)


torch.Size([16, 2])

In [ ]:
#hide
out = create_scripts()
beep(out)

<IPython.core.display.Javascript object>

Converted 000_utils.ipynb.
Converted 000b_data.validation.ipynb.
Converted 000c_data.preparation.ipynb.
Converted 001_data.external.ipynb.
Converted 002_data.core.ipynb.
Converted 003_data.preprocessing.ipynb.
Converted 003b_data.transforms.ipynb.
Converted 003c_data.mixed_augmentation.ipynb.
Converted 003d_data.image.ipynb.
Converted 003e_data.features.ipynb.
Converted 005_data.tabular.ipynb.
Converted 006_data.mixed.ipynb.
Converted 007_metrics.ipynb.
Converted 008_learner.ipynb.
Converted 009_optimizer.ipynb.
Converted 010_callback.core.ipynb.
Converted 011_callback.noisy_student.ipynb.
Converted 012_callback.gblend.ipynb.
Converted 013_callback.TSBERT.ipynb.
Converted 100_models.utils.ipynb.
Converted 100b_models.layers.ipynb.
Converted 100c_models.explainability.ipynb.
Converted 101_models.ResNet.ipynb.
Converted 101b_models.ResNetPlus.ipynb.
Converted 102_models.InceptionTime.ipynb.
Converted 102b_models.InceptionTimePlus.ipynb.
Converted 103_models.MLP.ipynb.
Converted 103b_mode